In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch


d:\Program\Anaconda\envs\machine-learning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# IMDb 데이터셋 로드
dataset = load_dataset("imdb")

# 훈련 및 테스트 데이터셋 분리
train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))  # 1000개 샘플로 축소
test_dataset = dataset['test'].shuffle(seed=42).select(range(500))  # 500개 샘플로 축소

In [3]:
# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 데이터셋 토크나이징 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# 데이터셋 토크나이징 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 모델 입력으로 사용하기 위해 데이터셋 포맷 설정
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [4]:
# BERT 모델 로드
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_steps=10_000,
    save_total_limit=2,
)

# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 모델 훈련
trainer.train()
trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Program\Anaconda\envs\machine-learning\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/375 [00:00<?, ?it/s]d:\Program\Anaconda\envs\machine-learning\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
                                                 
 33%|███▎      | 125/3

{'eval_loss': 0.3331996202468872, 'eval_runtime': 8.7776, 'eval_samples_per_second': 56.963, 'eval_steps_per_second': 7.177, 'epoch': 1.0}


                                                 
 67%|██████▋   | 250/375 [02:13<00:58,  2.13it/s]

{'eval_loss': 0.35246196389198303, 'eval_runtime': 9.2922, 'eval_samples_per_second': 53.809, 'eval_steps_per_second': 6.78, 'epoch': 2.0}


                                                 
100%|██████████| 375/375 [03:24<00:00,  1.84it/s]


{'eval_loss': 0.4441220164299011, 'eval_runtime': 9.7368, 'eval_samples_per_second': 51.352, 'eval_steps_per_second': 6.47, 'epoch': 3.0}
{'train_runtime': 204.298, 'train_samples_per_second': 14.684, 'train_steps_per_second': 1.836, 'train_loss': 0.2821788533528646, 'epoch': 3.0}


100%|██████████| 63/63 [00:09<00:00,  6.81it/s]


{'eval_loss': 0.4441220164299011,
 'eval_runtime': 9.2597,
 'eval_samples_per_second': 53.998,
 'eval_steps_per_second': 6.804,
 'epoch': 3.0}

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score

# 평가 지표 함수 정의
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)  # 예측된 클래스
    labels = p.label_ids  # 실제 레이블
    acc = accuracy_score(labels, preds)  # 정확도 계산
    return {'accuracy': acc}

# 이미 훈련된 트레이너에 compute_metrics를 추가하여 평가
trainer.compute_metrics = compute_metrics

# 모델 평가 및 정확도 확인
eval_result = trainer.evaluate()
print(f"Accuracy: {eval_result['eval_accuracy']:.4f}")

100%|██████████| 63/63 [00:09<00:00,  6.72it/s]

Accuracy: 0.9020
